In [7]:
from sklearn import tree
import pandas as pd

In [83]:
# import reference into pandas dataframe

'''
0 : none
1 : Johannes
2 : Andrieu
3 : Hugo
4 : Benjamin
'''

# Hypothesis : The data is labelled + key are strings (surrounded by '')
files = ["hugo", "benjamin"]
dataframes = []
for file in files:
    df_file = pd.read_csv("test/"+file+".csv", sep=";", encoding = "ISO-8859-1")
    df_file["name"] = file
    dataframes.append(df_file)
df = pd.concat(dataframes)
    
key_map = dict(enumerate(df['Key'].astype('category').cat.categories))
key_mapping = {v: k for k, v in key_map.items()}
print(key_mapping)
df['key_numerical'] = df['Key'].map(key_mapping)

label_map = dict(enumerate(df['name'].astype('category').cat.categories))
label_mapping = {v: k for k, v in label_map.items()}
print(label_mapping)
df['label'] = df['name'].map(label_mapping)
#df["key_numerical"] = df["Key"].astype('category').cat.codes
#df["label"] = df["name"].astype('category').cat.codes

df.fillna(0)
df


{"'": 0, '\'"\'': 1, "'&'": 2, "','": 3, "'-'": 4, "'.'": 5, "'B'": 6, "'C'": 7, "'F'": 8, "'H'": 9, "'J'": 10, "'K'": 11, "'L'": 12, "'M'": 13, "'P'": 14, "'S'": 15, "'V'": 16, "'W'": 17, "'X'": 18, "'Y'": 19, "'\\x01'": 20, "'\\x13'": 21, "'a'": 22, "'b'": 23, "'c'": 24, "'d'": 25, "'e'": 26, "'f'": 27, "'g'": 28, "'h'": 29, "'i'": 30, "'j'": 31, "'k'": 32, "'l'": 33, "'m'": 34, "'n'": 35, "'o'": 36, "'p'": 37, "'q'": 38, "'r'": 39, "'s'": 40, "'t'": 41, "'u'": 42, "'v'": 43, "'w'": 44, "'x'": 45, "'y'": 46, "'z'": 47, "'è'": 48, "'é'": 49, "'ï¿½'": 50, 'Key.backspace': 51, 'Key.ctrl_l': 52, 'Key.enter': 53, 'Key.shift': 54, 'Key.space': 55, "['^']": 56}
{'benjamin': 0, 'hugo': 1}


,Unnamed: 0,Key,Shift,HD_mean,HD_std,DL_mean,DL_std,name,key_numerical,label
0,0,Key.shift,1,0.347687,0.220647,0.347687,0.222158,hugo,54,1
1,1,'P',1,0.091400,0.018084,0.091400,0.035752,hugo,14,1
2,2,'o',0,0.092588,0.028136,0.092588,0.166226,hugo,36,1
3,3,'r',0,0.095229,0.027034,0.095229,0.111068,hugo,39,1
4,4,'t',0,0.082236,0.018584,0.082236,0.181403,hugo,41,1
...,...,...,...,...,...,...,...,...,...,...
64,64,Key.backspace,0,0.092500,0.008500,0.092500,0.266500,benjamin,51,0
65,65,'s',0,0.063000,0.000000,0.063000,0.000000,benjamin,40,0
66,66,Key.ctrl_l,0,0.192000,0.000000,0.192000,0.000000,benjamin,52,0
67,67,'\x01',0,0.114000,0.000000,0.114000,0.000000,benjamin,20,0


In [82]:
features = ["key_numerical", "Shift", "HD_mean", "HD_std", "DL_mean", "DL_std"]
target = "label"


Y = df_hugo[target]
X = df_hugo[features]

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [9]:
clf.predict([[2., 2.]])

array([1])

In [10]:
clf.predict_proba([[2., 2.]])

array([[0., 1.]])